In [2]:
import numpy as np
from sklearn.neighbors import KernelDensity
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats as stats
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression


In [7]:
data = "data/train.csv"
df = pd.read_csv(data)

df.isna().values.any()


False

In [8]:
from sklearn.preprocessing import StandardScaler



In [9]:
# random forest regressor
kf = KFold(n_splits=10)
random_forest_regressor = RandomForestRegressor(n_estimators=10)
kng = KNeighborsRegressor(n_neighbors=4)
models = [('Random Forest Regressor', random_forest_regressor), ('k-Nearest Neighbors', kng),('Linear Regression', LinearRegression())]
in_sample_error = []
out_sample_error = []
for name, model in models:
    in_sample_err = []
    out_sample_err = []
    mse=[]
    for train, test in kf.split(X):
        model.fit(X.iloc[train], np.ravel(y.iloc[train]))
        model.fit(X.iloc[train], np.ravel(y.iloc[train]))
        predict_train = model.predict(X.iloc[train])
        predict_test = model.predict(X.iloc[test])
        predict = model.predict(X)
        mse.append((np.ravel(predict) - np.ravel(y))**2) 
        in_sample_err.append(MSE(predict_train, y.iloc[train]))
        out_sample_err.append(MSE(predict_test, y.iloc[test]))
    in_sample_error.append(in_sample_err)
    out_sample_error.append(out_sample_err)
    error = np.mean(mse, axis=0)
    zscore = (error-np.mean(error))/np.std(error)
    
    kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(zscore.reshape(-1,1))
    # create a set of x-values at which to evaluate the KDE
    x_vals = np.linspace(np.min(error), np.max(error), 1000).reshape(-1,1)
    # get the corresponding y-values by evaluating the KDE at the x-values 
    y_vals = np.exp(kde.score_samples(x_vals))
    # plot the KDE
    plt.title(name)
    plt.plot(x_vals, y_vals)
    plt.xlabel('error')
    plt.ylabel('Density')
    plt.show()
    print(name,'in sample error:',np.mean(in_sample_err))
    print(name,'out sample error:',np.mean(out_sample_err))
print('in sample t-Test')
print('Random Forest Regressor and k-Nearest Neighbors',stats.ttest_rel(in_sample_error[0], in_sample_error[1]))
print('Random Forest Regressor and Linear Regression',stats.ttest_rel(in_sample_error[0], in_sample_error[2]))
print('k-Nearest Neighbors and Linear Regression',stats.ttest_rel(in_sample_error[1], in_sample_error[2]))
print('out sample t-Test')
print('Random Forest Regressor and knn',stats.ttest_rel(out_sample_error[0], out_sample_error[1]))
print('Random Forest Regressor and Linear Regression',stats.ttest_rel(out_sample_error[0], out_sample_error[2]))
print('k-Nearest Neighbors and Linear Regression',stats.ttest_rel(out_sample_error[1], out_sample_error[2]))

NameError: name 'X' is not defined